In [1]:
import numpy as np
from numpy import genfromtxt

file_path = 'pre/train.csv'

# load dataset
def load_data(path):
    
    # exclude the row of title 
    num = len(open(path,'r').readlines()) - 1
    data = np.empty((num,4),dtype="uint32")
    label = np.empty((num,1),dtype="uint8")
    
    #label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecom,age,gender,residence
    arr = genfromtxt(path, delimiter=',', dtype="uint32", skip_header=1, usecols=(0,3,4,5,6,7))
    
    label = arr[:,0]
    data = arr[:,1:]
    
    
    return data,label

X,Y = load_data(file_path)


#preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
#print('Predicted:', decode_predictions(preds, top=3)[0])

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

train_csv = np.genfromtxt('pre/train.csv', dtype=float, delimiter=',', names=True, usecols=[0,1,3,4,5,6,7])
train_csv = np.sort(train_csv, order='clickTime')
test_csv = np.genfromtxt('pre/test.csv', dtype=float, delimiter=',', names=True, usecols=[0,1,3,4,5,6,7])
test_csv = np.sort(train_csv, order='clickTime')

user_csv = np.genfromtxt('pre/user.csv', dtype=float, delimiter=',', names=True)

t1 = pd.DataFrame(train_csv)
t2 = pd.DataFrame(user_csv)
t3 = pd.DataFrame(test_csv)

trainData = t1.merge(t2)
testData = t3.merge(t2)

#np.savetxt('train.txt',trainData)
#np.savetxt('test.txt',testData)

In [14]:
tmp = trainData.as_matrix()
X = tmp[:, [x for x  in (1,2,3,4,5)]].astype(np.uint32)
Y = tmp[:, 0].astype(np.uint32)

In [19]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_minmax = min_max_scaler.fit_transform(X)

/home/huyssen/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [20]:
train_split = 0.9

train_size = int(X_minmax.shape[0]*train_split)
test_size = X_minmax.shape[0] - train_size

X_train, X_test = X_minmax[0:train_size,:], X_minmax[train_size:,:]
y_train, y_test = Y[0:train_size], Y[train_size:,]

#clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
#max_depth=1, random_state=0).fit(X_train, y_train)

#clf.score(X_test, y_test) 
#y_pred = clf.predict(X_test)
index_train = np.where(y_train > 0)
index_test = np.where(y_test > 0)

print(X_train.shape)
print(X_test.shape)

(3374575, 5)
(374953, 5)


In [21]:
import xgboost as xgb


data_train, data_test = X_train, X_test
label_trian, label_test = y_train, y_test

#get data
dtrain = xgb.DMatrix(data_train, label=label_trian, missing = -999.0)
dtest = xgb.DMatrix(data_test, label=label_test, missing = -999.0)
#set params
params = {'bst:max_depth':2, 'bst:eta':1, 'silent':False, 'objective':'binary:logistic','nthread':6,'eval_metric':'logloss'}

evallist  = [(dtrain,'train'),(dtest,'eval')]
#train
num_round = 300
stopping_rounds = 15
bst = xgb.train(params, dtrain, num_round, evallist, early_stopping_rounds=stopping_rounds)
#15 step 0.09

/home/huyssen/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[0]	train-logloss:0.461862	eval-logloss:0.462917
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 10 rounds.
[1]	train-logloss:0.33549	eval-logloss:0.337156
[2]	train-logloss:0.257648	eval-logloss:0.257952
[3]	train-logloss:0.207313	eval-logloss:0.208437
[4]	train-logloss:0.173762	eval-logloss:0.1736
[5]	train-logloss:0.151383	eval-logloss:0.15224
[6]	train-logloss:0.136356	eval-logloss:0.137376
[7]	train-logloss:0.12634	eval-logloss:0.127617
[8]	train-logloss:0.119593	eval-logloss:0.121825
[9]	train-logloss:0.11516	eval-logloss:0.117898
[10]	train-logloss:0.112391	eval-logloss:0.114741
[11]	train-logloss:0.110588	eval-logloss:0.113337
[12]	train-logloss:0.109385	eval-logloss:0.112771
[13]	train-logloss:0.108618	eval-logloss:0.112676
[14]	train-logloss:0.107982	eval-logloss:0.112658
[15]	train-logloss:0.107295	eval-logloss:0.112473
[16]	train-logloss:0.106865	eval-logloss:0.112242
[17]	train-loglos

In [21]:
bst.save_model('size90_250_best.model') 0.085451

In [23]:
from sklearn import metrics
#print(data_test.shape)
y_pred = bst.predict(dtest)
print(y_pred)
print(label_test)
metrics.log_loss(label_test,y_pred)

[ 0.0055593   0.00099833  0.00926953 ...,  0.01527131  0.00622515
  0.00470881]
[0 0 0 ..., 0 0 0]


0.085235244974789445

In [22]:
# load real dataset
def load_pre_data(path):
    
    # exclude the row of title 
    num = len(open(path,'r').readlines()) - 1
    data = np.empty((num,4),dtype="uint32")
    instance = np.empty((num,1),dtype="uint32")
    
    arr = genfromtxt(path, delimiter=',', dtype="uint32", skip_header=1, usecols=(0,3,4,6,7))
    
    instance = arr[:,0]
    data = arr[:,1:]

    return instance,data

ins,realX = load_pre_data("pre/test.csv")


In [24]:
dreal = xgb.DMatrix(realX, missing = -999.0)
Y_real = bst.predict(dreal)

In [25]:
res = np.column_stack((ins,Y_real))
np.savetxt("pred5.csv", res,fmt='%.9f',delimiter=',',header=" instanceID,prob")
#print(res)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

def MLP(dim):


    model = Sequential()
    model.add(Dense(64, input_dim=dim, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    
    return model


model = MLP(6)

model.fit(X_train, y_train,epochs=20,batch_size=2500)

#score = model.evaluate(X_test, y_test, batch_size=128)

#plot_model(model, to_file='model.png')
#SVG(model_to_dot(model).create(prog='dot', format='svg'))